In [ ]:
import json
import getpass
from requests import Session
from pprint import pprint
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

url = "https://localhost"
user = input()
password = getpass.getpass()

# LOGIN

In [ ]:
session = Session()

first = session.get(f"{url}", verify=False)
csrftoken = first.cookies["csrftoken"]

data = json.dumps(
    {"username": user, "password": password, "csrfmiddlewaretoken": csrftoken}
)

headers = {
    "X-CSRFToken": first.headers["Set-Cookie"].split("=")[1].split(";")[0],
    "Referer": url,
    "X-Requested-With": "XMLHttpRequest",
}

req = session.post(
    f"{url}/api/auth/", data=data, cookies=first.cookies, headers=headers, verify=False
)
if req.status_code != 200:
    print(req.text)
    exit(1)

# GET RULE LIST

In [ ]:
rules = session.get(f"{url}/api/rules/?start=0&length=1&draw=0", verify=False).json()
print(f"{len(rules['data'])} rules returned of {rules['recordsTotal']}")
if len(rules['data']) > 0: 
    pprint(rules['data'][0])
    rule_pk = rules['data'][0]['id']

# CREATE RULE

In [ ]:

files = [
    ('files', ('example1.yar', open('./example1.yara','rb'))),
    ('files', ('example2.yar', open('./example2.yara','rb')))
]
res = session.post(f"{url}/api/rules/", files=files, cookies=first.cookies, headers=headers, verify=False)
if res.status_code == 200:
    data = res.json() or []
    pprint(res.json())
    rule_pks = [x['id'] for x in data]
else:
    print(res.status_code, res.text)

# BUILD RULE

In [ ]:
data = {
    "rule_ids": rule_pks,
    "rulename": "combined_rule"
}
res = session.post(f"{url}/api/rules/build", json=data, cookies=first.cookies, headers=headers, verify=False)
if res.status_code == 200:
    pprint(res.json())
else:
    print(res.status_code, res.text)

# EDIT RULE

In [ ]:
data = {
        "text": "rule NewRule { condition: true }"
    }
res = session.patch(f"{url}/api/rules/{rule_pks[0]}",  json=data, cookies=first.cookies, headers=headers, verify=False)
if res.status_code == 200:
    pprint(res.json())
else:
    print(res.status_code, res.text)

# DOWNLOAD RULE

In [ ]:
res = session.get(f"{url}/api/rules/{rule_pks[0]}/download", cookies=first.cookies, headers=headers, verify=False)
if res.status_code == 200:
    pprint(res.text)
else:
    print(res.status_code, res.text)

# DELETE RULES

In [ ]:
data = {
    "rule_ids": rule_pks
}
res = session.delete(f"{url}/api/rules/", json=data, cookies=first.cookies, headers=headers, verify=False)
print(res.status_code, res.text)   